## Decisions Tree

In [ ]:
def train_decision_tree(X, Y, num_nodes_stop=1, criterion='accuracy'):
    """ Returns a decision tree trained on X and Y. 
    Stops splitting nodes when a node has hit a size of "num_nodes_stop" or lower.
    Split criterion can be either 'accuracy' or 'entropy'.
    Returns a tree (In whatever format that you find appropriate)
    """
    tree ={}
    init_val = -1
    tree[0] = [init_val]*3
    split_node_dec(tree, 0, X, Y, 0, num_nodes_stop, criterion)
    return tree

def eval_decision_tree(tree, test_X):
    """ Takes in a tree, and a bunch of instances X and 
    returns the tree predicted values at those instances."""
    n = test_X.shape[0]
    Y_pred = np.zeros(n)
    for i in range(n):
        Y_pred[i] = pred_class(tree, 0, test_X[i])
    return Y_pred
    
neg = 0
pos = 0

def split_node_dec(tree, node_pos, X, Y, depth, num_nodes_stop=1, criterion = 'accuracy', stop_depth = 1e7):
    global neg, pos
    n,d = X.shape
    total_neg = np.sum((Y==-1))
    total_pos = np.sum((Y==1))
    one_class = False
    if total_pos == 0 or total_pos == n:
        one_class = True
    if n <= num_nodes_stop or depth >= stop_depth or ((pos == total_pos) and (neg == total_neg)) or one_class:
        if total_pos > total_neg:
            class_pred = 1
        else:
            class_pred = -1
        tree[node_pos] = [-1,-1, class_pred]
        return

    neg = total_neg
    pos = total_pos
    best_feat = 0
    best_thresh = 0
    best_acc = -np.inf
    
    for feat in range(d):
        ids = X[:,feat].argsort()
        X = X[ids]
        Y = Y[ids]
        min_feat = X[0][feat]
        max_feat = X[-1][feat]
        ### check here
        num_pts  = 11
        iter = max_feat - min_feat
        iter = iter/num_pts
        for pt in range(1, num_pts):
            thresh = min_feat + pt*iter
            pos_l = 0
            left = 0
            for sample in range(n):
                if X[sample][feat] <= thresh:
                    left+=1
                    if Y[sample]>0:
                        pos_l+=1
            pos_r = total_pos - pos_l
            right = n - left
            dec = split_dec(criterion, pos_l, left, pos_r, right)
            if dec >= best_acc:
                best_acc = dec
                best_feat = feat
                best_thresh = thresh
    X_left = X[X[:, best_feat] <= best_thresh]
    Y_left = Y[X[:, best_feat] <= best_thresh]
    X_right = X[X[:, best_feat] > best_thresh]
    Y_right = Y[X[:, best_feat] > best_thresh]
    tree[node_pos] = [best_feat, best_thresh, 0]
    split_node_dec(tree, 2*node_pos+1, X_left, Y_left, depth+1, num_nodes_stop, criterion, stop_depth)
    split_node_dec(tree, 2*node_pos+2, X_right, Y_right, depth+1, num_nodes_stop, criterion, stop_depth)
    return

def split_dec(criterion, pos_l, left, pos_r, right):
    if criterion == 'accuracy':
        val = (pos_l + right - pos_r)/(left+right)
        return max(val, 1-val)
    elif criterion == 'entropy':
        p_l = left/(left+right)
        h_l = h_func(pos_l/(left+1e-40))
        p_r = 1 - p_l
        h_r = h_func(pos_r/(right+1e-40))
        return (p_l*h_l + p_r*h_r)

def h_func(x):
    if x == 0 or x == 1:
        return 0
    return x*np.log2(x) + (1-x)*np.log2(1-x)

def pred_class(tree, node_pos, sample):
    if tree[node_pos][0] == -1:
        return tree[node_pos][2]
    loc = tree[node_pos][0]
    thresh = tree[node_pos][1]
    if sample[loc] <= thresh:
        return pred_class(tree, 2*node_pos+1, sample)
    else:
        return pred_class(tree, 2*node_pos+2, sample)
    
    
def train_valid_split(X_train, Y_train, split = 0.8):
    indices = [i for i in range(len(X_train))]
    train_indices = np.random.choice(indices, size=round(len(indices) * split), replace = False)
    valid_indices = list(set(indices) - set(train_indices))
    X_tr = []
    Y_tr = []
    X_valid = []
    Y_valid = []
    for index in train_indices:
        X_tr.append(X_train[index])
        Y_tr.append(Y_train[index])
    for index in valid_indices:
        X_valid.append(X_train[index])
        Y_valid.append(Y_train[index])
    return np.array(X_tr), np.array(Y_tr), np.array(X_valid), np.array(Y_valid)

def preprocess(file):
    data = np.load(file)
    X_train, Y_train, X_test, Y_test = np.array(data['arr_0']), np.array(data['arr_1']), np.array(data['arr_2']), np.array(data['arr_3'])
    #Y_train = np.vstack(Y_train)
    #Y_test = np.vstack(Y_test)
    mean, std = standardize(X_train)
    X_train = X_train-mean
    X_train = X_train/std
    X_test = X_test - mean
    X_test = X_test/std
    return X_train, Y_train, X_test, Y_test

def standardize(X):
    mean = np.mean(X, axis = 0)
    std = np.std(X, axis = 0)
    return mean, (std + 1e-20)

def choose_hyperparam(X_train, Y_train, X_test, Y_test, criterion):
    node_stop = [1, 2, 4, 8, 16, 32, 64, 128, 256]
    err = 1
    best_num_nodes_stop = 0
    for ns in node_stop:
        clf = train_decision_tree(X_train, Y_train, ns, criterion)
        Y_pred = eval_decision_tree(clf, X_test)
        test_err = np.sum(Y_pred != Y_test) / len(Y_pred)
        if test_err < err:
            err = test_err
            best_num_nodes_stop = ns
    return best_num_nodes_stop

def give_result(file, criteria):
    X_train, Y_train, X_test, Y_test = preprocess(file)
    # print(X_train.shape)
    # print(Y_train.shape)
    # print(X_test.shape)
    # print(Y_test.shape)
    X_tr, Y_tr, X_val, Y_val = train_valid_split(X_train, Y_train)
    results = []
    char1 = file[-5]
    for criterion in criteria:
        num_nodes_stop = choose_hyperparam(X_tr, Y_tr, X_val, Y_val, criterion)
        clf =  train_decision_tree(X_train, Y_train, num_nodes_stop, criterion)
        Y_pred_train = eval_decision_tree(clf, X_train)
        #print(Y_pred_train.shape)
        Y_pred_test = eval_decision_tree(clf, X_test)
        #print(Y_pred_test.shape)
        train_err = np.sum(Y_pred_train != Y_train) / len(Y_train)
        test_err = np.sum(Y_pred_test != Y_test) / len(Y_test)
        dict1 = {'Dataset':char1, 'Criterion': criterion,'num_nodes_stop': num_nodes_stop , 'Train loss': train_err, 'Test Loss': test_err}
        results.append(dict1)
    df1 = pd.DataFrame(results)
    return df1

## Random Forest Classifier

In [1]:
def train_random_forest(X, Y, num_trees=10, num_nodes_stop=1, 
                        criterion='accuracy', a=0.5, b=0.5):
    """ Returns a random forest trained on X and Y. 
    Trains num_trees.
    Stops splitting nodes in each tree when a node has hit a size of "num_nodes_stop" or lower.
    Split criterion can be either 'accuracy' or 'entropy'.
    Fraction of data used per tree = a
    Fraction of features used in each node = b
    Returns a random forest (In whatever format that you find appropriate)
    """
    n,d = X.shape
    rf = []
    for i in range(num_trees):
        frac_data = np.sort(np.random.choice(range(n), int(n*a), replace = False))
        rf.append(train_dt_rf(X[frac_data], Y[frac_data], num_nodes_stop, criterion, b))
    return rf
    

def eval_random_forest(random_forest, test_X):
    """ Takes in a  random forest object (hhowever you want to store it), and a bunch of instances X and 
    returns the tree predicted values at those instances."""
    m, dim = test_X.shape
    Y_pred = np.zeros(m)
    for tree in random_forest:
        Y_pred = Y_pred + eval_dt_rf(tree, test_X)
    return np.sign(Y_pred)

def train_dt_rf(X, Y, num_nodes_stop, criterion, b):
    tree = {}
    init_val = -1
    tree[0] = [init_val]*3
    split_node_dec_rf(tree, 0, X, Y, 0, b, num_nodes_stop, criterion)
    return tree

def eval_dt_rf(tree, test_X):
    n = test_X.shape[0]
    Y = np.zeros(n)
    for i in range(n):
        Y[i] = pred_class_rf(tree, 0, test_X[i])
    return Y


def pred_class_rf(tree, node_pos, sample):
    if tree[node_pos][0] == -1:
        return tree[node_pos][2]
    loc = tree[node_pos][0]
    thresh = tree[node_pos][1]
    if sample[loc] <= thresh:
        return pred_class(tree, 2*node_pos+1, sample)
    else:
        return pred_class(tree, 2*node_pos+2, sample)


neg = 0
pos = 0

def split_node_dec_rf(tree, node_pos, X, Y, depth, b, num_nodes_stop=1, criterion = 'accuracy', stop_depth = 1e7):
    global neg, pos
    n,d = X.shape
    total_neg = np.sum((Y==-1))
    total_pos = np.sum((Y==1))
    one_class = False
    if total_pos == 0 or total_pos == n:
        one_class = True
    if n <= num_nodes_stop or depth >= stop_depth or ((pos == total_pos) and (neg == total_neg)) or one_class:
        if total_pos > total_neg:
            class_pred = 1
        else:
            class_pred = -1
        tree[node_pos] = [-1,-1, class_pred]
        return

    neg = total_neg
    pos = total_pos
    best_feat = 0
    best_thresh = 0
    best_acc = -np.inf
    feats = np.sort(np.random.choice(range(d), int(d*b), replace = False))
    for feat in feats:
        ids = X[:,feat].argsort()
        X = X[ids]
        Y = Y[ids]
        min_feat = X[0][feat]
        max_feat = X[-1][feat]
        ### check here
        num_pts  = 11
        iter = max_feat - min_feat
        iter = iter/num_pts
        for pt in range(1, num_pts):
            thresh = min_feat + pt*iter
            pos_l = 0
            left = 0
            for sample in range(n):
                if X[sample][feat] <= thresh:
                    left+=1
                    if Y[sample]>0:
                        pos_l+=1
            pos_r = total_pos - pos_l
            right = n - left
            dec = split_dec_rf(criterion, pos_l, left, pos_r, right)
            if dec >= best_acc:
                best_acc = dec
                best_feat = feat
                best_thresh = thresh
    X_left = X[X[:, best_feat] <= best_thresh]
    Y_left = Y[X[:, best_feat] <= best_thresh]
    X_right = X[X[:, best_feat] > best_thresh]
    Y_right = Y[X[:, best_feat] > best_thresh]
    tree[node_pos] = [best_feat, best_thresh, 0]
    split_node_dec_rf(tree, 2*node_pos+1, X_left, Y_left, depth+1, b, num_nodes_stop, criterion, stop_depth)
    split_node_dec_rf(tree, 2*node_pos+2, X_right, Y_right, depth+1, b, num_nodes_stop, criterion, stop_depth)
    return

def split_dec_rf(criterion, pos_l, left, pos_r, right):
    if criterion == 'accuracy':
        val = (pos_l + right - pos_r)/(left+right)
        return max(val, 1-val)
    elif criterion == 'entropy':
        p_l = left/(left+right)
        h_l = h_func(pos_l/(left+1e-40))
        p_r = 1 - p_l
        h_r = h_func(pos_r/(right+1e-40))
        return (p_l*h_l + p_r*h_r)

def h_func(x):
    if x == 0 or x == 1:
        return 0
    return x*np.log2(x) + (1-x)*np.log2(1-x)

def train_valid_split(X_train, Y_train, split = 0.8):
    indices = [i for i in range(len(X_train))]
    train_indices = np.random.choice(indices, size=round(len(indices) * split), replace = False)
    valid_indices = list(set(indices) - set(train_indices))
    X_tr = []
    Y_tr = []
    X_valid = []
    Y_valid = []
    for index in train_indices:
        X_tr.append(X_train[index])
        Y_tr.append(Y_train[index])
    for index in valid_indices:
        X_valid.append(X_train[index])
        Y_valid.append(Y_train[index])
    return np.array(X_tr), np.array(Y_tr), np.array(X_valid), np.array(Y_valid)

def preprocess(file):
    data = np.load(file)
    X_train, Y_train, X_test, Y_test = np.array(data['arr_0']), np.array(data['arr_1']), np.array(data['arr_2']), np.array(data['arr_3'])
    #Y_train = np.vstack(Y_train)
    #Y_test = np.vstack(Y_test)
    mean, std = standardize(X_train)
    X_train = X_train-mean
    X_train = X_train/std
    X_test = X_test - mean
    X_test = X_test/std
    return X_train, Y_train, X_test, Y_test

def standardize(X):
    mean = np.mean(X, axis = 0)
    std = np.std(X, axis = 0)
    return mean, (std + 1e-20)

def choose_hyperparam(X_train, Y_train, X_test, Y_test, criterion):
    num_trees = [5, 10, 20, 30, 40, 50, 60, 70, 80]
    err = 1
    best_num_trees = 0
    for nt in num_trees:
        clf = train_random_forest(X_train, Y_train, nt, 1, criterion)
        Y_pred = eval_random_forest(clf, X_test)
        test_err = np.sum(Y_pred != Y_test) / len(Y_pred)
        if test_err < err:
            err = test_err
            best_num_trees = nt
    return best_num_trees

def give_result(file, criteria):
    X_train, Y_train, X_test, Y_test = preprocess(file)
    # print(X_train.shape)
    # print(Y_train.shape)
    # print(X_test.shape)
    # print(Y_test.shape)
    X_tr, Y_tr, X_val, Y_val = train_valid_split(X_train, Y_train)
    results = []
    char1 = file[-5]
    for criterion in criteria:
        num_trees = choose_hyperparam(X_tr, Y_tr, X_val, Y_val, criterion)
        clf =  train_random_forest(X_train, Y_train, num_trees, 1, criterion)
        Y_pred_train = eval_random_forest(clf, X_train)
        #print(Y_pred_train.shape)
        Y_pred_test = eval_random_forest(clf, X_test)
        #print(Y_pred_test.shape)
        train_err = np.sum(Y_pred_train != Y_train) / len(Y_train)
        test_err = np.sum(Y_pred_test != Y_test) / len(Y_test)
        dict1 = {'Dataset':char1, 'Criterion': criterion,'Number of trees': num_trees , 'Train loss': train_err, 'Test Loss': test_err}
        results.append(dict1)
    df1 = pd.DataFrame(results)
    return df1

criteria = ['accuracy', 'entropy']
file_name = '/content/Data/dataset_D.npz'
## uncomment the following two blocks to see the most recurring hyperparams over 10 iterations
# i = 10
# while i > 0:
#     result = give_result(file_name, criteria)
#     print(result)
#     i-=1
## uncomment the following block of code for getting one iteration of results    
# result = give_result(file_name, criteria)
# result
X_train, Y_train, X_test, Y_test = preprocess('/content/Data/dataset_A.npz')
X_min = min(X_train[:,0]),min(X_train[:,1])
X_max = max(X_train[:,0]),max(X_train[:,1])
X,Y = np.meshgrid(np.arange(X_min[0]-0.5,X_max[0]+0.5,0.05),np.arange(X_min[1]-0.5,X_max[1]+0.5,0.05))
testing_set= np.concatenate([X.reshape(-1,1),Y.reshape(-1,1)],axis=1)
clf = train_random_forest(X_train, Y_train, num_trees = 30, num_nodes_stop = 1, criterion='accuracy')
predA = eval_random_forest(clf, testing_set)

plt.figure(0)
f, ax = plt.subplots(1,1,sharex=False,sharey=True,figsize=(12,8))
f.suptitle('Random Forest Classification on dataset A', size=18)
ax.contourf(X,Y,predA.reshape(X.shape), cmap = mpl.colors.ListedColormap(('skyblue', 'gold')), alpha=0.3)
ax.scatter(X_train[Y_train==1][:,0], X_train[Y_train==1][:,1], s=2,c='darkorange', label='Class 1')
ax.scatter(X_train[Y_train==-1][:,0], X_train[Y_train==-1][:,1], s=2, c='blue',label='Class -1')
ax.set_title(f'Plot when criterion is accuracy')
ax.set_xlabel(r'$x_{1}\rightarrow$',size=15)
ax.set_ylabel(r'$x_{2}\rightarrow$',size=15)
ax.legend()
plt.show()


X_train, Y_train, X_test, Y_test = preprocess('/content/Data/dataset_B.npz')
X_min = min(X_train[:,0]),min(X_train[:,1])
X_max = max(X_train[:,0]),max(X_train[:,1])
X,Y = np.meshgrid(np.arange(X_min[0]-0.5,X_max[0]+0.5,0.05),np.arange(X_min[1]-0.5,X_max[1]+0.5,0.05))
testing_set= np.concatenate([X.reshape(-1,1),Y.reshape(-1,1)],axis=1)
clf = train_random_forest(X_train, Y_train, num_trees = 50, num_nodes_stop = 1, criterion='accuracy')
predB = eval_random_forest(clf, testing_set)

plt.figure(0)
f, ax = plt.subplots(1,1,sharex=False,sharey=True,figsize=(12,8))
f.suptitle('Random Forest Classification on dataset B', size=18)
ax.contourf(X,Y,predB.reshape(X.shape), cmap = mpl.colors.ListedColormap(('skyblue', 'gold')), alpha=0.3)
ax.scatter(X_train[Y_train==1][:,0], X_train[Y_train==1][:,1], s=2,c='darkorange', label='Class 1')
ax.scatter(X_train[Y_train==-1][:,0], X_train[Y_train==-1][:,1], s=2, c='blue',label='Class -1')
ax.set_title(f'Plot when criterion is accuracy')
ax.set_xlabel(r'$x_{1}\rightarrow$',size=15)
ax.set_ylabel(r'$x_{2}\rightarrow$',size=15)
ax.legend()
plt.show()